In [11]:
import torch
import os
import json
import nibabel as nib
import numpy as np
from tqdm import tqdm
from PIL import Image
from llava.model.builder import load_pretrained_model
from llava.mm_utils import KeywordsStoppingCriteria, get_model_name_from_path, process_images, tokenizer_image_token
from llava.conversation import conv_templates, SeparatorStyle
from huggingface_hub import snapshot_download

In [14]:
MODEL_PATH="/home/hufsaim/VLM/VLM/m3/train/checkpoints/test3/checkpoint-798"
MODEL_BASE=None
SOURCE="local"

OUTPUT_PATH = "/home/hufsaim/VLM/VLM/m3/demo/0527"
JSON_FILE = "/home/hufsaim/VLM/VLM/data/0527/question.json"
SLICE_SAVE_PATH = "/home/hufsaim/VLM/VLM/m3/demo/sliced_images"

MODEL_CARDS = ""

"""Here is a list of available expert models:\n
<BRATS(args)> 
Modality: MRI, 
Task: segmentation, 
Overview: A pre-trained model for volumetric (3D) segmentation of brain tumor subregions from multimodal MRIs based on BraTS 2018 data, 
Accuracy: Tumor core (TC): 0.8559 - Whole tumor (WT): 0.9026 - Enhancing tumor (ET): 0.7905 - Average: 0.8518, 
Valid args are: None\n
<VISTA3D(args)> 
Modality: CT, 
Task: segmentation, 
Overview: domain-specialized interactive foundation model developed for segmenting and annotating human anatomies with precision, 
Accuracy: 127 organs: 0.792 Dice on average, 
Valid args are: 'everything', 'hepatic tumor', 'pancreatic tumor', 'lung tumor', 'bone lesion', 'organs', 'cardiovascular', 'gastrointestinal', 'skeleton', or 'muscles'\n
<VISTA2D(args)> 
Modality: cell imaging, 
Task: segmentation, 
Overview: model for cell segmentation, which was trained on a variety of cell imaging outputs, including brightfield, phase-contrast, fluorescence, confocal, or electron microscopy, 
Accuracy: Good accuracy across several cell imaging datasets, 
Valid args are: None\n
<CXR(args)> 
Modality: chest x-ray (CXR), 
Task: classification, 
Overview: pre-trained model which are trained on large cohorts of data, 
Accuracy: Good accuracy across several diverse chest x-rays datasets, 
Valid args are: None\n
<HD-Glio(args)>
Modality: MRI, 
Task: segmentation, 
Overview: A deep learning-based model designed for high-grade glioma segmentation in brain MR images. HD-Glio leverages ensemble 3D U-Net architectures and robust preprocessing including brain extraction, intensity normalization, and co-registration. It is tailored to identify and delineate tumor subregions (enhancing tumor, tumor core, and whole tumor) with high accuracy.
Accuracy: Tumor core (TC): 0.860 - Whole tumor (WT): 0.910 - Enhancing tumor (ET): 0.800 - Average: 0.857
Valid args are: None\n
Give the model <NAME(args)> when selecting a suitable expert model.\n"""

"Here is a list of available expert models:\n\n<BRATS(args)> \nModality: MRI, \nTask: segmentation, \nOverview: A pre-trained model for volumetric (3D) segmentation of brain tumor subregions from multimodal MRIs based on BraTS 2018 data, \nAccuracy: Tumor core (TC): 0.8559 - Whole tumor (WT): 0.9026 - Enhancing tumor (ET): 0.7905 - Average: 0.8518, \nValid args are: None\n\n<VISTA3D(args)> \nModality: CT, \nTask: segmentation, \nOverview: domain-specialized interactive foundation model developed for segmenting and annotating human anatomies with precision, \nAccuracy: 127 organs: 0.792 Dice on average, \nValid args are: 'everything', 'hepatic tumor', 'pancreatic tumor', 'lung tumor', 'bone lesion', 'organs', 'cardiovascular', 'gastrointestinal', 'skeleton', or 'muscles'\n\n<VISTA2D(args)> \nModality: cell imaging, \nTask: segmentation, \nOverview: model for cell segmentation, which was trained on a variety of cell imaging outputs, including brightfield, phase-contrast, fluorescence, co

In [13]:
def load_nifti_image(nifti_path, slice_axis=2, sample_id="default"):
    try:
        nifti_img = nib.load(nifti_path)
        volume = nifti_img.get_fdata()
    except Exception as e:
        raise RuntimeError(f"Failed to load NIfTI file: {nifti_path}, error: {e}")

    slice_idx = volume.shape[slice_axis] // 2

    if slice_axis == 0:
        slice_img = volume[slice_idx, :, :]
    elif slice_axis == 1:
        slice_img = volume[:, slice_idx, :]
    else:
        slice_img = volume[:, :, slice_idx]

    slice_norm = (slice_img - np.min(slice_img)) / (np.max(slice_img) - np.min(slice_img) + 1e-8)
    slice_image = Image.fromarray(np.uint8(slice_norm * 255)).convert('RGB')

    os.makedirs(SLICE_SAVE_PATH, exist_ok=True)
    save_path = os.path.join(SLICE_SAVE_PATH, f"{sample_id}.png")
    slice_image.save(save_path)

    return slice_image

In [4]:
def save_result_to_json(result, output_dir, base_filename="inference_result_wo_expert"):
    os.makedirs(output_dir, exist_ok=True)
    counter = 1

    filepath = os.path.join(output_dir, f"{base_filename}_{counter}.json")
    while os.path.exists(filepath):
        filepath = os.path.join(output_dir, f"{base_filename}_{counter}.json")
        counter += 1

    formatted_results = []
    for sample_id, info_list in result.items():
        combined = {"id": sample_id}
        for entry in info_list:
            combined.update(entry)
        formatted_results.append(combined)

    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(formatted_results, f, ensure_ascii=False, indent=4)


def load_json_data(json_path):
    with open(json_path, "r") as f:
        return json.load(f)

In [15]:
class M3Inference:
    def __init__(self, source=SOURCE, model_path=MODEL_PATH, conv_mode="llama_3"):
        self.source = source
        if source == "local" or source == "huggingface":
            # TODO: allow setting the device
            self.conv_mode = conv_mode
            if source == "huggingface":
                model_path = snapshot_download(model_path)
            model_name = get_model_name_from_path(model_path)
            self.tokenizer, self.model, self.image_processor, self.context_len = load_pretrained_model(
                model_path, model_name, MODEL_BASE
            )
        else:
            raise NotImplementedError(f"Source {source} is not supported.")

    def inference(self, image_path, prompt, sample_id="default"):
        answer = []
        conv = conv_templates[self.conv_mode].copy()
        answer.append({"USER": prompt})
        answer.append({"image path": image_path})

        if image_path.endswith(('.nii', '.nii.gz')):
            image = load_nifti_image(image_path, sample_id=sample_id)
        else:
            image = Image.open(image_path).convert('RGB')

        images_tensor = process_images([image], self.image_processor, self.model.config).to(self.model.device, dtype=torch.float16)
        full_prompt = f"{MODEL_CARDS}\n<image>\n{prompt}"

        media_input = {"image": [img for img in images_tensor]}
        media_config = {"image": {}}

        conv.append_message(conv.roles[0], full_prompt)
        conv.append_message(conv.roles[1], "")
        prompt_text = conv.get_prompt()
        input_ids = tokenizer_image_token(prompt_text, self.tokenizer, return_tensors="pt").unsqueeze(0).to(self.model.device)

        stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
        stopping_criteria = KeywordsStoppingCriteria([stop_str], self.tokenizer, input_ids)

        with torch.inference_mode():
            output_ids = self.model.generate(
                input_ids,
                media=media_input,
                media_config=media_config,
                max_new_tokens=1024,
                temperature=0.0,
                top_p=0.9,
                use_cache=True,
                stopping_criteria=[stopping_criteria],
                pad_token_id=self.tokenizer.eos_token_id,
            )

        output = self.tokenizer.batch_decode(output_ids, skip_special_tokens=True)[0].strip()
        if output.endswith(stop_str):
            output = output[:-len(stop_str)].strip()

        answer.append({"VILA-M3": output})
        return answer

In [16]:
def run_batch_inference(pairs):
    inference_result = {}
    inference_model = M3Inference()
    for pair in tqdm(pairs, desc="Processing Cases"):
        sample_id = pair["id"]
        image_path = pair["image_path"]
        question = pair["question"]

        if isinstance(image_path, list):
            image_path = image_path[0]

        inference_result[sample_id] = inference_model.inference(
            image_path=image_path,
            prompt=question,
            sample_id=sample_id
        )

    return inference_result

In [17]:
DATA_ROOT_DIR = "/home/hufsaim/VLM/VLM/data/0527"

if __name__ == "__main__":
    if JSON_FILE:
        data = load_json_data(JSON_FILE)
        pairs = []

        for sample in data:
            sample_id = sample["id"]
            image_path = sample["image path"]
            question = sample["USER"]

            if isinstance(image_path, list):
                image_path = [os.path.join(DATA_ROOT_DIR, p) if not os.path.isabs(p) else p for p in image_path]
            elif isinstance(image_path, str):
                if not os.path.isabs(image_path):
                    image_path = os.path.join(DATA_ROOT_DIR, image_path)
            else:
                raise TypeError(f"Unexpected image_path type: {type(image_path)} (id: {sample_id})")

            pairs.append({"id": sample_id, "image_path": image_path, "question": question})

        results = run_batch_inference(pairs=pairs)
        save_result_to_json(results, OUTPUT_PATH)
    else:
        raise ValueError("JSON_FILE path must be specified.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Processing Cases:   0%|          | 0/40 [00:00<?, ?it/s]/home/hufsaim/anaconda3/envs/vila/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/hufsaim/anaconda3/envs/vila/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Processing Cases: 100%|██████████| 40/40 [00:07<00:00,  5.10it/s]
